# [문제 1] 서울 소재 31개의 경찰서의 크롤링

In [103]:
# 【문항01】 경찰 민원포탈 > 경찰관서 찾기 > 목록에서 찾기 > 서울에서 서울 소재 31개의 경찰서의 이름과 주소를 크롤링하여 데이터프레임을 만드세요. (40점)
# (단, 셀레니움을 이용해서 가져오지 않고 다른 사이트를 이용해서 가져오면 감점 20점)

In [167]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from urllib.parse import quote
import numpy as np
import time
import pandas as pd
import requests

### 1-1.데이터 가져오기 (10점)

In [84]:
url = 'https://minwon.police.go.kr/#customerCenter/plcstSearch'
driver = webdriver.Chrome()
driver.get(url)

In [87]:
#목록에서 찾기로 넘어가가기
driver.find_element_by_css_selector('#tab3_1').click()

In [88]:
#지역에서 서울 선택하기
driver.find_element_by_css_selector('.check').click()

### 1-2.원하는 데이터를 찾고, 하나를 선택해서 원하는 정보 추출하기 (10점)

- 첫번째 페이지

In [89]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [90]:
trs = soup.select('#minwonTbody>tr')
len(trs)

5

In [91]:
tr = trs[0]
tr

<tr> <td>경찰청 서울특별시경찰청 서울중부경찰서 </td>
<td class="title">서울특별시 중구 수표로 27</td>
<td>
<a addr="서울특별시 중구 수표로 27" href="#" id="viewMap" latd="37.563630778711754" lotd="126.99002154542026" plcstid="3" plcstnm="경찰청 서울특별시경찰청 서울중부경찰서">
<img alt="지도보기" src="/images/icon/icon_map.gif"/>
</a>
</td></tr>

In [92]:
pname = tr.find('td').get_text().strip()
pname = pname.split()[-1]
pname

'서울중부경찰서'

In [93]:
ploc = tr.select_one('.title').get_text().strip()
ploc

'서울특별시 중구 수표로 27'

In [94]:
lines= []
for tr in trs:
    pname = tr.td.get_text()
    ploc = tr.select_one('.title').get_text()
    lines.append([pname, ploc])



In [ ]:
lines

[['경찰청 서울특별시경찰청 서울중부경찰서 ', '서울특별시 중구 수표로 27'],
 ['경찰청 서울특별시경찰청 서울종로경찰서 ', '서울특별시 종로구 율곡로 46'],
 ['경찰청 서울특별시경찰청 서울남대문경찰서 ', '서울특별시 중구 한강대로 410'],
 ['경찰청 서울특별시경찰청 서울서대문경찰서 ', '서울특별시 서대문구 통일로 113'],
 ['경찰청 서울특별시경찰청 서울혜화경찰서 ', '서울특별시 종로구 창경궁로 112-16']]

In [ ]:
df = pd.DataFrame(lines)
df


,0,1
0,경찰청 서울특별시경찰청 서울중부경찰서,서울특별시 중구 수표로 27
1,경찰청 서울특별시경찰청 서울종로경찰서,서울특별시 종로구 율곡로 46
2,경찰청 서울특별시경찰청 서울남대문경찰서,서울특별시 중구 한강대로 410
3,경찰청 서울특별시경찰청 서울서대문경찰서,서울특별시 서대문구 통일로 113
4,경찰청 서울특별시경찰청 서울혜화경찰서,서울특별시 종로구 창경궁로 112-16


### 1-3.반복문을 사용하여 모든 페이지의 데이터 가져오기 및 오류 수정하기 (10점)

In [98]:
police_list= []
for _ in range(6):
    driver.find_element_by_css_selector('.p.next').click()
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('#minwonTbody > tr')
    for tr in trs:
        pname = tr.find('td').get_text().strip().split()[-1]
        ploc = tr.select_one('.title').get_text()
        police_list.append([pname, ploc])



In [100]:
driver.close()

### 1-4.데이터 프레임으로 만들고 저장하기 (10점)

In [141]:
df2 = pd.DataFrame(police_list,columns=['경찰서명','주소'])
df2

,경찰서명,주소
0,서울용산경찰서,서울특별시 용산구 원효로89길 24
1,서울성북경찰서,서울특별시 성북구 보문로 170
2,서울동대문경찰서,서울특별시 동대문구 약령시로 21길 29
3,서울마포경찰서,서울특별시 마포구 마포대로 183
4,서울영등포경찰서,서울특별시 영등포구 국회대로 608
5,서울성동경찰서,서울특별시 성동구 왕십리광장로 9
6,서울동작경찰서,서울특별시 동작구 노량진로 148
7,서울광진경찰서,서울특별시 광진구 광나루로 447 광진소방서 임시청사 (능동)
8,서울서부경찰서,서울특별시 은평구 은평로9길 15
9,서울강북경찰서,서울특별시 강북구 오패산로 406


- 오류나는 주소 수정

In [153]:
# 서울서부경찰서 서울특별시 은평구 서오릉로 58
# 서울강동경찰서 서울 강동구 양재대로96길 35
df2.주소[8] = '서울특별시 은평구 서오릉로 58'
df2.주소[15] = '서울 강동구 양재대로96길 35'


In [154]:
df2

,경찰서명,주소
0,서울용산경찰서,서울특별시 용산구 원효로89길 24
1,서울성북경찰서,서울특별시 성북구 보문로 170
2,서울동대문경찰서,서울특별시 동대문구 약령시로 21길 29
3,서울마포경찰서,서울특별시 마포구 마포대로 183
4,서울영등포경찰서,서울특별시 영등포구 국회대로 608
5,서울성동경찰서,서울특별시 성동구 왕십리광장로 9
6,서울동작경찰서,서울특별시 동작구 노량진로 148
7,서울광진경찰서,서울특별시 광진구 광나루로 447 광진소방서 임시청사 (능동)
8,서울서부경찰서,서울특별시 은평구 서오릉로 58
9,서울강북경찰서,서울특별시 강북구 오패산로 406


In [155]:
df2.to_csv('서울경찰서위치.csv',index=False)

# [문제 2] 카카오 API를 이용해 위도 경도 가져오기

In [104]:
# 【문항02】1번에서 구한 경찰서의 도로명 주소를 가지고, 카카오로컬 API를 이용하여 경찰서의 위도, 경도 정보를 구한 후 데이터프레임에 추가하세요. (30점)

### 2-1. 카카오 로컬 API 사이트에 접속하기 (10점)

In [162]:
#kakaoapikey.txt 복사해서 해당 파일에 붙혀넣음
with open('kakaoapikey.txt') as file:
    kakao_key = file.read()

local_url = "https://dapi.kakao.com/v2/local/search/address.json"

addr = pd.read_csv('서울경찰서위치.csv')

header = {'Authorization' : f'KakaoAK {kakao_key}'}

### 2-2. 반복문을 사용하여 모든 경찰서의 위도, 경도 좌표 구하기 (10점)

In [163]:
lng_list, lat_list = [], []
for i in addr.index:
    url = f'{local_url}?query={quote(addr.주소[i])}'
    result = requests.get(url, headers=header).json()
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(addr.경찰서명[i], addr.주소[i])


### 2-3. 데이터프레임에 위도, 경도 컬럼 추가하기 (10점)

In [170]:
addr['위도'] = lat_list
addr['경도'] = lng_list
addr.head(3)

,경찰서명,주소,위도,경도
0,서울용산경찰서,서울특별시 용산구 원효로89길 24,37.541538,126.967648
1,서울성북경찰서,서울특별시 성북구 보문로 170,37.589980,127.016215
2,서울동대문경찰서,서울특별시 동대문구 약령시로 21길 29,37.585158,127.045794


In [171]:
addr

,경찰서명,주소,위도,경도
0,서울용산경찰서,서울특별시 용산구 원효로89길 24,37.541538,126.967648
1,서울성북경찰서,서울특별시 성북구 보문로 170,37.589980,127.016215
2,서울동대문경찰서,서울특별시 동대문구 약령시로 21길 29,37.585158,127.045794
3,서울마포경찰서,서울특별시 마포구 마포대로 183,37.550830,126.954018
4,서울영등포경찰서,서울특별시 영등포구 국회대로 608,37.525956,126.901006
5,서울성동경찰서,서울특별시 성동구 왕십리광장로 9,37.562003,127.036199
6,서울동작경찰서,서울특별시 동작구 노량진로 148,37.512796,126.942549
7,서울광진경찰서,서울특별시 광진구 광나루로 447 광진소방서 임시청사 (능동),37.545698,127.079137
8,서울서부경찰서,서울특별시 은평구 서오릉로 58,37.605846,126.923591
9,서울강북경찰서,서울특별시 강북구 오패산로 406,37.637170,127.027042


# [문제 3] 서울지도위에 표시

In [ ]:
# 【문항03】 1,2번에서 구한 정보를 바탕으로 서울 지도(Open Street Map)위에 표시하세요. (30점)

### 3-1. 서울 지도를 그릴 것 (10점)

In [168]:
import folium
map = folium.Map(location=[37.559868, 126.967109], zoom_start=14)
map

### 3-2. 툴팁에는 경찰서명을 표시하고, 팝업에는 경찰서 주소를 표시하되 팝업창의 너비를 늘려 가로로 표시할 것 (10점)

In [172]:
addr.head()

,경찰서명,주소,위도,경도
0,서울용산경찰서,서울특별시 용산구 원효로89길 24,37.541538,126.967648
1,서울성북경찰서,서울특별시 성북구 보문로 170,37.589980,127.016215
2,서울동대문경찰서,서울특별시 동대문구 약령시로 21길 29,37.585158,127.045794
3,서울마포경찰서,서울특별시 마포구 마포대로 183,37.550830,126.954018
4,서울영등포경찰서,서울특별시 영등포구 국회대로 608,37.525956,126.901006


In [177]:
map = folium.Map(location=[addr.위도.mean(), addr.경도.mean()], zoom_start=12)
for i in addr.index:
    folium.Marker(
        location=[addr.위도[i], addr.경도[i]],
        popup=folium.Popup(addr.주소[i], max_width=200),
        tooltip=addr.경찰서명[i]
    ).add_to(map)
    
map

### 3-3. 아이콘은 기본 모양이 아닌 것으로 표시하고, 지도 제목을 ‘서울소재 경찰서 위치’로 표시할 것 (10점)

In [175]:
map = folium.Map(location=[addr.위도.mean(), addr.경도.mean()], zoom_start=12)
for i in addr.index:
    folium.Marker(
        location=[addr.위도[i], addr.경도[i]],
        popup=folium.Popup(addr.주소[i], max_width=200),
        tooltip=addr.경찰서명[i],
        icon=folium.Icon(icon='glyphicon-glass')
    ).add_to(map)

title_html = '<h3 align="center" style="font-size:40px">서울소재 경찰서 위치</h3>'    
map.get_root().html.add_child(folium.Element(title_html))
map
   